In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import torch
from gluonnlp.data import SentencepieceTokenizer
from model.net import KobertCRF
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

In [2]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids, line=0):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        # ----------- parsing list_of_ner_word and decoding_ner_sentence ----------- #
        list_of_ner_word = []
        decoding_ner_sentence = ""
        
        entity_tag, prev_entity_tag = "", ""
        prev_bio_class = 'O'
        
        for token_str, pred_ner_tag_str in list(zip(input_token, pred_ner_tag))[1:]:
            token_str = token_str.replace('▁', ' ')# '▁' 토큰을 띄어쓰기로 교체
            if token_str == '[SEP]':
                if prev_bio_class == 'I':   #text가 I로 끝나는 경우 닫기
                    decoding_ner_sentence += f':{prev_entity_tag}>'
                break

            if "B-" in pred_ner_tag_str:
                if prev_bio_class != 'O':
                    decoding_ner_sentence += f':{prev_entity_tag}>' 
                
                entity_tag = pred_ner_tag_str[-3:]

                list_of_ner_word.append({"word": token_str, "tag": entity_tag})
                if token_str[0] == ' ':
                    decoding_ner_sentence += token_str.replace(' ',' <')
                else:
                    decoding_ner_sentence += f'<{token_str}'

                prev_entity_tag = entity_tag
                prev_bio_class = 'B'

            elif "I-"+prev_entity_tag in pred_ner_tag_str:
                if prev_bio_class != 'O':
                    list_of_ner_word[-1]['word'] += token_str
                    decoding_ner_sentence += token_str

                    prev_bio_class = 'I'
                else:   # I로 시작하는 경우 B로 간주함
                    entity_tag = pred_ner_tag_str[-3:]
                    
                    list_of_ner_word.append({"word": token_str, "tag": entity_tag})
                    if token_str[0] == ' ':
                        decoding_ner_sentence += token_str.replace(' ',' <')
                    else:
                        decoding_ner_sentence += f'<{token_str}'

                    prev_entity_tag = entity_tag
                    prev_bio_class = 'B'

            else:
                if prev_bio_class != 'O':
                    decoding_ner_sentence += f':{prev_entity_tag}>'
                decoding_ner_sentence += token_str

                prev_bio_class = 'O'
                entity_tag, prev_entity_tag = "", ""
            
        return list_of_ner_word, decoding_ner_sentence

In [3]:
def main(input_texts):
    model_dir = Path('./experiments/base_model_with_crf')
    model_config = Config(json_path=model_dir / 'config.json')

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # load vocab & tokenizer
    tok_path = "./ptr_lm_model/tokenizer_78b3253a26.model"
    ptr_tokenizer = SentencepieceTokenizer(tok_path)

    with open(model_dir / "vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)
    tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=model_config.maxlen)

    # load ner_to_index.json
    with open(model_dir / "ner_to_index.json", 'rb') as f:
        ner_to_index = json.load(f)
        index_to_ner = {v: k for k, v in ner_to_index.items()}

    # model
    model = KobertCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)

    # load
    model_dict = model.state_dict()
    checkpoint = torch.load("./experiments/base_model_with_crf/best-epoch-16-step-1500-acc-0.993.bin", map_location=device)
    # checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))
    convert_keys = {}
    for k, v in checkpoint['model_state_dict'].items():
        new_key_name = k.replace("module.", '')
        if new_key_name not in model_dict:
            print("{} is not int model_dict".format(new_key_name))
            continue
        convert_keys[new_key_name] = v

    model.load_state_dict(convert_keys)
    model.eval()
    model.to(device)
    decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)

    for input_text in input_texts:
        print(f'input> {input_text}')
        
        list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([input_text])
        x_input = torch.tensor(list_of_input_ids).long().to(device)
        list_of_pred_ids = model(x_input)

        list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
        print(f"output> {decoding_ner_sentence}", end='\n\n')

In [4]:
input_texts = [
"지난달 28일 수원에 살고 있는 윤주성 연구원은 코엑스(서울 삼성역)에서 개최되는 DEVIEW 2019 Day1에 참석했다. LaRva팀의 '엄~청 큰 언어 모델 공장 가동기!' 세션을 들으며 언어모델을 학습시킬때 multi-GPU, TPU 모두 써보고 싶다는 생각을 했다.",
"문재인 대통령은 28일 서울 코엑스에서 열린 ‘데뷰 (Deview) 2019’ 행사에 참석해 젊은 개발자들을 격려하면서 우리 정부의 인공지능 기본구상을 내놓았다.  출처 : 미디어오늘 (http://www.mediatoday.co.kr)",
"SKTBrain에서 KoBERT 모델을 공개해준 덕분에 BERT-CRF 기반 개체명인식기를 쉽게 개발할 수 있었다.",
"터미네이터: 다크 페이트 (Terminator: Dark Fate)는 2019년 개봉한 미국의 SF, 액션영화이다. 1991년 영화 터미네이터 2: 심판의 날 이후 28년 만에 제임스 카메론이 제작자로서 시리즈에 복귀한 작품이다. 린다 해밀턴이 사라 코너 역으로 돌아오면서 아널드 슈워제네거와 함께 주연을 맡았다.",
"[뉴스토마토 김희경 기자] 영화 '터미네이터: 다크 페이트'(감독 팀 밀러)가 박스오피스 1위는 물론 전체 예매율 1위를 차지했다. 시리즈 최고 오프닝 스코어 경신과 함께 겹경사다.",
"전 세계 최고의 기대작 <어벤져스> 시리즈의 압도적 대미를 장식할 <어벤져스: 엔드게임>이 지난 4월 14일(일)과 15일(월) 양일간 진행된 대한민국 내한 행사를 성공적으로 마무리 지었다. <어벤져스: 엔드게임>의 주역 로버트 다우니 주니어, 제레미 레너, 브리 라슨, 안소니 루소&조 루소 감독, 트린 트랜 프로듀서, 케빈 파이기 마블 스튜디오 대표까지 방문하여 특별한 대한민국 사랑을 뽐냈다.",
"영화 '겨울왕국2'의 이현민 애니메이션 슈퍼바이저가 SBS '나이트라인'에 출연해 다양한 이야기를 전했다. 숨겨진 과거의 비밀과 새로운 운명을 찾기 위해 모험을 떠나는 '엘사'와 '안나'의 이야기를 담은 작품이다.",
"네이버(NAVER (167,000원▲ 3,000 1.83%)) 금융 계열사인 ‘네이버파이낸셜’이 1일 출범했다. 네이버파이낸셜은 기존 결제·송금 서비스를 하던 ‘네이버페이’를 분사해 설립한 회사다. 네이버파이낸셜은 외연을 확장해 ‘네이버 통장’을 선보이고, 이어 주식, 보험, 예·적금, 신용카드 서비스도 출시한다는 계획이다.",
]

main(input_texts)

input> 지난달 28일 수원에 살고 있는 윤주성 연구원은 코엑스(서울 삼성역)에서 개최되는 DEVIEW 2019 Day1에 참석했다. LaRva팀의 '엄~청 큰 언어 모델 공장 가동기!' 세션을 들으며 언어모델을 학습시킬때 multi-GPU, TPU 모두 써보고 싶다는 생각을 했다.
output>  지난달 <28일:DAT> <수원:LOC>에 살고 있는 <윤주성:PER> 연구원은 <코엑스:LOC>(<서울:LOC> <삼성역:LOC>)에서 개최되는 <DEVIEW 2019 Day1:POH>에 참석했다. <LaRva팀:ORG>의 '엄~청 큰 언어 모델 공장 가동기!' 세션을 들으며 언어모델을 학습시킬때 multi-GPU, TPU 모두 써보고 싶다는 생각을 했다.

input> 문재인 대통령은 28일 서울 코엑스에서 열린 ‘데뷰 (Deview) 2019’ 행사에 참석해 젊은 개발자들을 격려하면서 우리 정부의 인공지능 기본구상을 내놓았다.  출처 : 미디어오늘 (http://www.mediatoday.co.kr)
output>  <문재인:PER> 대통령은 <28일:DAT> <서울 코엑스:LOC>에서 열린 ‘<데뷰 (Deview) 2019:POH>’ 행사에 참석해 젊은 개발자들을 격려하면서 우리 정부의 인공지능 기본구상을 내놓았다. 출처 : <미디어오늘:POH> (<http://www.mediatoday.co.kr:POH>)

input> SKTBrain에서 KoBERT 모델을 공개해준 덕분에 BERT-CRF 기반 개체명인식기를 쉽게 개발할 수 있었다.
output>  <SKTBrain:ORG>에서 <KoBERT:POH> 모델을 공개해준 덕분에 <BERT-CRF:POH> 기반 개체명인식기를 쉽게 개발할 수 있었다.

input> 터미네이터: 다크 페이트 (Terminator: Dark Fate)는 2019년 개봉한 미국의 SF, 액션영화이다. 1991년 영화 터미네이터 2: 심판의 날 이후 28년 만에 제임스 카메론이 제작자로서 시리즈에 복귀한 작품이다. 린다 해밀턴이 